# Análise de Dados Olímpicos - Processamento Bronze Layer

Este notebook realiza:
1. Leitura dos dados raw (CSV)
2. Conversão para formato Parquet
3. Salvamento na camada Bronze
4. Criação de metadados

In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

## 1. Configuração de Paths

In [ ]:
# Definir paths
BASE_PATH = Path('.')
RAW_PATH = BASE_PATH / 'raw'
BRONZE_PATH = BASE_PATH / 'bronze'
GOLD_PATH = BASE_PATH / 'gold'
OUTPUT_PATH = BASE_PATH / 'outputs'

# Criar diretórios se não existirem
BRONZE_PATH.mkdir(exist_ok=True)
GOLD_PATH.mkdir(exist_ok=True)
(OUTPUT_PATH / 'tables').mkdir(parents=True, exist_ok=True)
(OUTPUT_PATH / 'figures').mkdir(parents=True, exist_ok=True)

print("✓ Estrutura de diretórios criada")

## 2. Função para Criar Metadados

In [ ]:
def create_metadata(df, dataset_name, source_file, description):
    """Cria metadados para um dataset"""
    metadata = {
        "dataset_name": dataset_name,
        "source_file": source_file,
        "description": description,
        "created_at": datetime.now().isoformat(),
        "shape": {
            "rows": len(df),
            "columns": len(df.columns)
        },
        "columns": list(df.columns),
        "dtypes": df.dtypes.astype(str).to_dict(),
        "missing_values": df.isnull().sum().to_dict(),
        "memory_usage_mb": round(df.memory_usage(deep=True).sum() / 1024**2, 2)
    }
    return metadata

def save_metadata(metadata, filepath):
    """Salva metadados em arquivo JSON"""
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    print(f"✓ Metadados salvos: {filepath}")

## 3. Processamento Olympics 1896-2022

In [ ]:
print("\n" + "="*80)
print("PROCESSANDO: Olympics 1896-2022")
print("="*80)

olympics_path = RAW_PATH / 'Olympics_1896_2022'

### 3.1 Game (Informações dos Jogos)

In [ ]:
print("\n[1/6] Processando: game...")
df_game = pd.read_csv(olympics_path / 'world_olympedia_olympics_game.csv')

# Criar coluna para identificar verão/inverno
df_game['season'] = df_game['edition'].apply(lambda x: 'Winter' if 'Winter' in str(x) else 'Summer')

print(f"  Shape: {df_game.shape}")
print(f"  Colunas: {df_game.columns.tolist()}")

# Salvar em parquet
output_file = BRONZE_PATH / 'game.parquet'
df_game.to_parquet(output_file, index=False)
print(f"✓ Salvo: {output_file}")

# Criar e salvar metadados
metadata = create_metadata(df_game, 'Olympic Games', 
                          'world_olympedia_olympics_game.csv',
                          'Informações sobre cada edição dos Jogos Olímpicos')
save_metadata(metadata, BRONZE_PATH / 'game_metadata.json')

df_game.head()

### 3.2 Country (Países)

In [ ]:
print("\n[2/6] Processando: country...")
df_country = pd.read_csv(olympics_path / 'world_olympedia_olympics_country.csv')

print(f"  Shape: {df_country.shape}")
print(f"  Colunas: {df_country.columns.tolist()}")

# Salvar em parquet
output_file = BRONZE_PATH / 'country.parquet'
df_country.to_parquet(output_file, index=False)
print(f"✓ Salvo: {output_file}")

# Criar e salvar metadados
metadata = create_metadata(df_country, 'Countries', 
                          'world_olympedia_olympics_country.csv',
                          'Lista de países participantes com códigos NOC')
save_metadata(metadata, BRONZE_PATH / 'country_metadata.json')

df_country.head()

### 3.3 Medal Tally (Contagem de Medalhas)

In [ ]:
print("\n[3/6] Processando: medal_tally...")
df_medal_tally = pd.read_csv(olympics_path / 'world_olympedia_olympics_game_medal_tally.csv')

# Adicionar informação de temporada
df_medal_tally = df_medal_tally.merge(
    df_game[['edition', 'season']], 
    on='edition', 
    how='left'
)

print(f"  Shape: {df_medal_tally.shape}")
print(f"  Colunas: {df_medal_tally.columns.tolist()}")

# Salvar em parquet
output_file = BRONZE_PATH / 'medal_tally.parquet'
df_medal_tally.to_parquet(output_file, index=False)
print(f"✓ Salvo: {output_file}")

# Criar e salvar metadados
metadata = create_metadata(df_medal_tally, 'Medal Tally', 
                          'world_olympedia_olympics_game_medal_tally.csv',
                          'Contagem de medalhas por país e edição dos jogos')
save_metadata(metadata, BRONZE_PATH / 'medal_tally_metadata.json')

df_medal_tally.head()

### 3.4 Athlete Bio (Biografia dos Atletas)

In [ ]:
print("\n[4/6] Processando: athlete_bio...")
df_athlete_bio = pd.read_csv(olympics_path / 'world_olympedia_olympics_athlete_bio.csv')

print(f"  Shape: {df_athlete_bio.shape}")
print(f"  Colunas: {df_athlete_bio.columns.tolist()}")

# Salvar em parquet (compressão ajuda com arquivo grande)
output_file = BRONZE_PATH / 'athlete_bio.parquet'
df_athlete_bio.to_parquet(output_file, index=False, compression='snappy')
print(f"✓ Salvo: {output_file}")

# Criar e salvar metadados
metadata = create_metadata(df_athlete_bio, 'Athlete Biography', 
                          'world_olympedia_olympics_athlete_bio.csv',
                          'Informações biográficas dos atletas olímpicos')
save_metadata(metadata, BRONZE_PATH / 'athlete_bio_metadata.json')

df_athlete_bio.head()

### 3.5 Athlete Event Result (Resultados por Atleta/Evento)

In [ ]:
print("\n[5/6] Processando: athlete_event_result...")
df_athlete_event = pd.read_csv(olympics_path / 'world_olympedia_olympics_athlete_event_result.csv')

# Adicionar informação de temporada
df_athlete_event = df_athlete_event.merge(
    df_game[['edition', 'season', 'year']], 
    on='edition', 
    how='left'
)

print(f"  Shape: {df_athlete_event.shape}")
print(f"  Colunas: {df_athlete_event.columns.tolist()}")

# Salvar em parquet
output_file = BRONZE_PATH / 'athlete_event_result.parquet'
df_athlete_event.to_parquet(output_file, index=False, compression='snappy')
print(f"✓ Salvo: {output_file}")

# Criar e salvar metadados
metadata = create_metadata(df_athlete_event, 'Athlete Event Results', 
                          'world_olympedia_olympics_athlete_event_result.csv',
                          'Resultados detalhados de atletas por evento')
save_metadata(metadata, BRONZE_PATH / 'athlete_event_result_metadata.json')

df_athlete_event.head()

### 3.6 Result (Resultados dos Eventos)

In [ ]:
print("\n[6/6] Processando: result...")
df_result = pd.read_csv(olympics_path / 'world_olympedia_olympics_result.csv')

print(f"  Shape: {df_result.shape}")
print(f"  Colunas: {df_result.columns.tolist()}")

# Salvar em parquet
output_file = BRONZE_PATH / 'result.parquet'
df_result.to_parquet(output_file, index=False)
print(f"✓ Salvo: {output_file}")

# Criar e salvar metadados
metadata = create_metadata(df_result, 'Event Results', 
                          'world_olympedia_olympics_result.csv',
                          'Resultados detalhados dos eventos olímpicos')
save_metadata(metadata, BRONZE_PATH / 'result_metadata.json')

df_result.head()

## 4. Processamento Olympics Paris 2024

In [ ]:
print("\n" + "="*80)
print("PROCESSANDO: Olympics Paris 2024")
print("="*80)

paris_path = RAW_PATH / 'Olympics_Paris2024'

### 4.1 Athletes Paris 2024

In [ ]:
print("\n[1/3] Processando: athletes_paris2024...")
df_athletes_2024 = pd.read_csv(paris_path / 'athletes.csv')

print(f"  Shape: {df_athletes_2024.shape}")
print(f"  Colunas: {df_athletes_2024.columns.tolist()}")

# Salvar em parquet
output_file = BRONZE_PATH / 'athletes_paris2024.parquet'
df_athletes_2024.to_parquet(output_file, index=False)
print(f"✓ Salvo: {output_file}")

# Criar e salvar metadados
metadata = create_metadata(df_athletes_2024, 'Athletes Paris 2024', 
                          'athletes.csv',
                          'Informações dos atletas participantes das Olimpíadas de Paris 2024')
save_metadata(metadata, BRONZE_PATH / 'athletes_paris2024_metadata.json')

df_athletes_2024.head()

### 4.2 Medals Paris 2024

In [ ]:
print("\n[2/3] Processando: medals_paris2024...")
df_medals_2024 = pd.read_csv(paris_path / 'medals.csv')

print(f"  Shape: {df_medals_2024.shape}")
print(f"  Colunas: {df_medals_2024.columns.tolist()}")

# Salvar em parquet
output_file = BRONZE_PATH / 'medals_paris2024.parquet'
df_medals_2024.to_parquet(output_file, index=False)
print(f"✓ Salvo: {output_file}")

# Criar e salvar metadados
metadata = create_metadata(df_medals_2024, 'Medals Paris 2024', 
                          'medals.csv',
                          'Medalhas conquistadas nas Olimpíadas de Paris 2024')
save_metadata(metadata, BRONZE_PATH / 'medals_paris2024_metadata.json')

df_medals_2024.head()

### 4.3 Medals Total Paris 2024

In [ ]:
print("\n[3/3] Processando: medals_total_paris2024...")
df_medals_total_2024 = pd.read_csv(paris_path / 'medals_total.csv')

print(f"  Shape: {df_medals_total_2024.shape}")
print(f"  Colunas: {df_medals_total_2024.columns.tolist()}")

# Salvar em parquet
output_file = BRONZE_PATH / 'medals_total_paris2024.parquet'
df_medals_total_2024.to_parquet(output_file, index=False)
print(f"✓ Salvo: {output_file}")

# Criar e salvar metadados
metadata = create_metadata(df_medals_total_2024, 'Medals Total Paris 2024', 
                          'medals_total.csv',
                          'Total de medalhas por país nas Olimpíadas de Paris 2024')
save_metadata(metadata, BRONZE_PATH / 'medals_total_paris2024_metadata.json')

df_medals_total_2024.head()

## 5. Resumo do Processamento Bronze

In [ ]:
print("\n" + "="*80)
print("RESUMO DO PROCESSAMENTO")
print("="*80)

bronze_files = list(BRONZE_PATH.glob('*.parquet'))
print(f"\nTotal de arquivos Parquet criados: {len(bronze_files)}")
print("\nArquivos:")
for file in sorted(bronze_files):
    size_mb = file.stat().st_size / 1024**2
    print(f"  - {file.name:40s} ({size_mb:8.2f} MB)")

metadata_files = list(BRONZE_PATH.glob('*_metadata.json'))
print(f"\nTotal de arquivos de metadados criados: {len(metadata_files)}")

print("\n✓ Camada Bronze processada com sucesso!")